In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rec').getOrCreate()

In [2]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [3]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [4]:
data.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [5]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [7]:
training, test = data.randomSplit([0.8,0.2])

In [8]:
als = ALS(maxIter=5,regParam=0.01,userCol='userId',
          itemCol='movieId',ratingCol='rating')

In [9]:
model = als.fit(training)

In [10]:
predictions = model.transform(test)

In [11]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    27| 0.11682114|
|     31|   1.0|    13|  2.3705938|
|     31|   1.0|     4|  2.9157841|
|     31|   1.0|    29|  1.0868318|
|     31|   3.0|    14|  2.5222964|
|     31|   1.0|     0|-0.10761018|
|     85|   5.0|    16|  1.7240696|
|     85|   2.0|    20|  0.9425014|
|     85|   1.0|     4|   2.286122|
|     85|   5.0|     8|   2.222345|
|     53|   1.0|     6|  3.1293578|
|     78|   1.0|    12| 0.46962342|
|     78|   1.0|     4| 0.94309825|
|     34|   1.0|    15| -1.3080249|
|     34|   1.0|    14|  2.6492124|
|     81|   5.0|    28| 0.19049251|
|     81|   3.0|    26|  2.0698137|
|     81|   1.0|    16|  1.2693247|
|     81|   1.0|     7|  1.1532795|
|     28|   1.0|    27|  1.3503937|
+-------+------+------+-----------+
only showing top 20 rows



In [14]:
predictions[predictions['userId'] == 7].show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     81|   1.0|     7| 1.1532795|
|      3|   1.0|     7| 0.3667051|
|     54|   1.0|     7|  2.006818|
|      4|   1.0|     7| 2.7088463|
|     39|   3.0|     7| 1.9676267|
|     77|   1.0|     7| 1.1297472|
|     21|   1.0|     7| 1.3805662|
|     75|   1.0|     7| 1.1019887|
|     42|   2.0|     7| 1.4242435|
+-------+------+------+----------+



In [15]:
predictions.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)
 |-- prediction: float (nullable = true)



In [16]:
evaluator = RegressionEvaluator(metricName='rmse',
                                labelCol='rating',
                               predictionCol='prediction')

In [17]:
rmse = evaluator.evaluate(predictions)

In [18]:
print('RMSE')
print(rmse)

RMSE
1.9582756649915483


In [19]:
single_user = test.filter(test['userId']==11).select('movieId','userId')

In [20]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     10|    11|
|     35|    11|
|     39|    11|
|     45|    11|
|     47|    11|
|     50|    11|
|     59|    11|
|     66|    11|
|     77|    11|
|     80|    11|
|     82|    11|
|     89|    11|
|     90|    11|
+-------+------+



In [21]:
recommendations = model.transform(single_user)

In [22]:
recommendations.orderBy('prediction',ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     90|    11| 6.4281373|
|     10|    11| 5.5816555|
|     50|    11| 2.7509756|
|     89|    11| 2.6614244|
|     59|    11| 2.5048614|
|     39|    11| 2.1225896|
|     77|    11| 1.7630403|
|     80|    11| 1.7450991|
|     66|    11|  1.103372|
|     45|    11| 0.5647117|
|     35|    11|0.39690918|
|     47|    11|-1.8095531|
|     82|    11|-2.7121654|
+-------+------+----------+

